In [2]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2
import psycopg2.extras
import json
import os

data_path = "."

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        HOST       = db_conn_dict['host']
        DB_USER    = db_conn_dict['user']
        DB_PW      = db_conn_dict['password']
        DEFAULT_DB = db_conn_dict['user']

        try:
            db = create_engine('postgres+psycopg2://'+DB_USER+':'+DB_PW+'@'+HOST+'/'+DEFAULT_DB, echo=False)
            conn = db.connect()
            print('connected')
        except Exception as e:
            print("unable to connect to the database")
            print(e)
        return db,conn

credfilepath = os.path.join(data_path, "data2x02_db.json")
db, conn = pgconnect(credfilepath)

print(db.table_names())

connected
['spatial_ref_sys', 'rfsnsw_bfpl', 'sa2_2016', 'organisations', 'Stonks', 'statisticalareas', 'businessstats', 'neighbourhoods']


We are creating a table for the BusinessStats.csv

In [4]:
conn.execute("DROP TABLE IF EXISTS businessstats")

business_stats = """CREATE TABLE IF NOT EXISTS businessstats (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         number_of_businesses Integer,
                         accomodation_and_food_services Integer,
                         retail_trade Integer,
                         agriculture_forestry_and_fishing Integer,
                         health_care_and_social_assistance Integer,
                         public_administration_and_safety Integer,
                         transport_postal_and_warehousing Integer
                   )"""
conn.execute(business_stats)
print("Created table for business stats")

Created table for business stats


In [3]:
business_stats_file = pd.read_csv('BusinessStats.csv')

table_name = "businessstats"
business_stats_file.to_sql(table_name, con=conn, if_exists='replace',index=False)

#Testing if table has been created
print ("Business Stats table created")
res = pd.read_sql_query('SELECT * FROM businessstats', conn)
res


Business Stats table created


,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
...,...,...,...,...,...,...,...,...,...
2296,901011001,Christmas Island,0,0,0,0,0,0,0
2297,901021002,Cocos (Keeling) Islands,7,3,0,0,0,0,0
2298,901031003,Jervis Bay,6,0,3,0,0,0,0
2299,901041004,Norfolk Island,0,0,0,0,0,0,0


In [5]:
# Creating a table for neighbourhoods
conn.execute("DROP TABLE IF EXISTS neighbourhoods")

neighbourhoods = """CREATE TABLE IF NOT EXISTS neighbourhoods (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         land_area Integer,
                         population Integer,
                         number_of_dwellings Integer,
                         number_of_businesses Integer,
                         median_annual_household_income Integer,
                         avg_monthly_rent Integer
                   )"""
conn.execute(neighbourhoods)
print(db.table_names())

['spatial_ref_sys', 'organisations', 'Stonks', 'statisticalareas', 'businessstats', 'neighbourhoods']


Creating a table for Neighbourhoods.csv

In [6]:
neighbourhoods_file = pd.read_csv('Neighbourhoods.csv')

#Testing if table has been created
table_name = "neighbourhoods"
neighbourhoods_file.to_sql(table_name, con=conn, if_exists='replace',index=False)

res = pd.read_sql_query('SELECT * FROM neighbourhoods', conn)
res

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent
0,102011028,Avoca Beach - Copacabana,643.8000,7590,2325,738.0,46996.0,1906.0
1,102011029,Box Head - MacMasters Beach,3208.6000,10986,3847,907.0,42621.0,1682.0
2,102011030,Calga - Kulnura,76795.1000,4841,1575,1102.0,42105.0,1182.0
3,102011031,Erina - Green Point,3379.3000,14237,4450,1666.0,43481.0,1595.0
4,102011032,Gosford - Springfield,1691.2000,19385,6373,2126.0,45972.0,1382.0
...,...,...,...,...,...,...,...,...
317,106011109,Cessnock Region,1570.4341,"7,931","3,281",673.0,73164.0,1080.0
318,106011113,Singleton Region,4067.2349,"4,919","2,055",698.0,87984.0,1000.0
319,111021218,Morisset - Cooranbong,330.5208,"14,959","6,298",1154.0,58084.0,1260.0
320,114021285,Hill Top - Colo Vale,174.3752,"6,025","2,249",400.0,81120.0,1512.0


In [12]:
import pandas as pd

df = pd.read_excel('BurntAreas.xlsx')
print (df)



       last_date           location_name  location_id location_type metric_id  \
0     2020-06-26   Armidale Regional (A)        10130           LGA       NaN   
1     2020-06-26             Ballina (A)        10250           LGA       NaN   
2     2020-06-26         Bega Valley (A)        10550           LGA       NaN   
3     2020-06-26           Bellingen (A)        10600           LGA       NaN   
4     2020-06-26      Blue Mountains (C)        10900           LGA       NaN   
...          ...                     ...          ...           ...       ...   
4203  2021-02-22  Northern Grampians (S)        25810           LGA     6_101   
4204  2021-02-22  Southern Grampians (S)        26260           LGA     6_101   
4205  2021-02-22              Towong (S)        26670           LGA     6_101   
4206  2021-02-22         Wangaratta (RC)        26700           LGA     6_101   
4207  2021-02-22          Wellington (S)        26810           LGA     6_101   

         category          